In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#benign = 1 and malignant = 0
dataset = pd.read_csv('Breast_cancer_data[1].csv')
dataset = dataset.sort_values(by='mean_area')
dataset.head(10)

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
101,6.981,13.43,43.79,143.5,0.11700,1
539,7.691,25.44,48.34,170.4,0.08668,1
538,7.729,25.49,47.98,178.8,0.08098,1
568,7.760,24.54,47.92,181.0,0.05263,1
46,8.196,16.84,51.71,201.9,0.08600,1
151,8.219,20.70,53.27,203.9,0.09405,1
314,8.597,18.60,54.09,221.2,0.10740,1
525,8.571,13.10,54.53,221.3,0.10360,1
61,8.598,20.98,54.66,221.8,0.12430,1
59,8.618,11.79,54.34,224.5,0.09752,1


In [3]:
dataset.isnull().sum()

mean_radius        0
mean_texture       0
mean_perimeter     0
mean_area          0
mean_smoothness    0
diagnosis          0
dtype: int64

In [4]:
# stores all the column labels in the list except for last column
# which is our target variable.
m= dataset.shape[1]
header = dataset.columns
header = list(header)
print(header)
dataset.shape

['mean_radius', 'mean_texture', 'mean_perimeter', 'mean_area', 'mean_smoothness', 'diagnosis']


(569, 6)

In [5]:
# dividing the dataset into 75% for training and 25% for testing 
idx = 3*dataset.shape[0]//4
train = dataset.iloc[:idx,:]
test = dataset.iloc[idx:,:]
print("last 5 rows of training dataset:\n\n",train.tail())
print("\nfirst 5 rows of testing dataset:\n\n",test.head())

last 5 rows of training dataset:

      mean_radius  mean_texture  mean_perimeter  mean_area  mean_smoothness  \
133        15.71         13.93           102.0      761.7          0.09462   
182        15.70         20.31           101.2      766.6          0.09597   
258        15.66         23.20           110.2      773.5          0.11090   
11         15.78         17.89           103.6      781.0          0.09710   
118        15.78         22.91           105.7      782.6          0.11550   

     diagnosis  
133          1  
182          0  
258          0  
11           0  
118          0  

first 5 rows of testing dataset:

      mean_radius  mean_texture  mean_perimeter  mean_area  mean_smoothness  \
13         15.85         23.95           103.7      782.7          0.08401   
375        16.17         16.07           106.3      788.5          0.09880   
330        16.03         15.51           105.8      793.2          0.09491   
10         16.02         23.24           102.7

In [6]:
train = np.array(train)
test = np.array(test)
print(train.shape[0])
# train[:,-1].astype(int)
# test[:,-1].astype(int)
# print(train,end='\n\n')
# print(test)

426


In [25]:
def unique_values(dataset,col_index):
    unique = np.unique(dataset[:,col_index])
    return(unique)
    
#section is a portion of the dataset for which we find entropy
def count_label(section):
    cnt = {}
    for row in dataset:
        if row[-1] not in cnt:
            cnt[row[-1]]=0
        cnt[row[-1]]+=1
    return cnt


def entropy(section):
    cnt = count_label(section)
    total = 0
    for val in cnt.values():
        total += val
        
    ent = 0 #entropy value
    for val in cnt.values():
        ent += (val/total)*np.log(val/total)
    
    return -1*ent

# dataset_ent is the entropy of the entire dataset 
def info_gain(left,right,parent_ent):
    n = len(left) + len(right)
    
    left_ent=entropy(left)
    right_ent=entropy(right)
    
    weighted_avg = (len(left)/n)*left_ent + (len(right)/n)*right_ent
    
    return parent_ent-weighted_avg
    
    
# partitions the dataset according to the question value.
# it compares each value of the colum at col_index and 
# with the question value and seperates them into two groups 
# left and right.
def partition(dataset,qval,col_index):
    n = dataset.shape[0]
    right=[]
    left=[]
    for i in range(n):
        if dataset[i,col_index]>=qval:
            right.append(dataset[i,:])
        else:
            left.append(dataset[i,:])
    
    #left = np.array(left)
    #left = np.delete(left,col_index,1)
    
    #right = np.array(right)
    #right = np.delete(right,col_index,1)

    return left,right


# finds the best split by iterating over each and every value
# in each and every column. It finds a suitable value for questioning
# and then finds the information gain of that attribute based on that question.
# best_attr stores the column label used for question
# best_ques stores the value using which we could find the best split.

def best_split(dataset,header,column):
    maxGain=0
    best_ques=None
    best_attr=None
    best_left=[]
    best_right=[]
    rows = dataset.shape[0]
    cols = dataset.shape[1]
    parent_ent = entropy(dataset)
    
    #we will see the best question value, except for the last column
    for c in range(cols-1):
        if c in column:
            continue
        for r in range(rows):
            left,right=partition(dataset,dataset[r,c],c)
            IG = info_gain(left,right,parent_ent)
            if maxGain<IG:
                maxGain = IG
                best_ques = dataset[r,c]
                best_attr = c
                best_left = left
                best_right = right
            
    best_left=np.array(best_left)
    best_right=np.array(best_right)
    
    return best_ques,best_attr,best_left,best_right

def splitting(ds,itr,header,space,col):
    if itr==0 or len(col)==len(header)-1:
        print('\n-----------------------------------------------------------------------------------\n')
        return
    
    ques,attr,left,right = best_split(ds,header,col)
    col.append(attr)
    
    
    if attr is not None:
#         print(f"\n{header[attr]}:\n",ds)
        
        print("\n"+'-'*space+"> "+header[attr]+" < "+ques.astype(str),end='')
        #print(ds.shape,"l")
        cnt = unique_values(left,ds.shape[1]-1)
        if cnt.shape[0]>1:
            splitting(left,itr-1,header,space+2,col)
        else:
            print(" ---> stop , predicted value is ",cnt[0])
            print("\n\n",left)
    
        print("\n"+'-'*space+"> "+header[attr]+" >= "+ques.astype(str),end='')
        cnt = unique_values(right,ds.shape[1]-1)
        #print(ds.shape,"r")
        if cnt.shape[0]>1:
            splitting(right,itr-1,header,space+2,col)
        else:
            print(" ---> stop , predicted value is ",cnt[0])
            print("\n\n",right)
            
        
    
    

In [26]:
itr = 10
space = 1
col=[]
splitting(train,itr,header,space,col)


-> mean_radius < 9.405 ---> stop , predicted value is  1.0


 [[6.981e+00 1.343e+01 4.379e+01 1.435e+02 1.170e-01 1.000e+00]
 [7.691e+00 2.544e+01 4.834e+01 1.704e+02 8.668e-02 1.000e+00]
 [7.729e+00 2.549e+01 4.798e+01 1.788e+02 8.098e-02 1.000e+00]
 [7.760e+00 2.454e+01 4.792e+01 1.810e+02 5.263e-02 1.000e+00]
 [8.196e+00 1.684e+01 5.171e+01 2.019e+02 8.600e-02 1.000e+00]
 [8.219e+00 2.070e+01 5.327e+01 2.039e+02 9.405e-02 1.000e+00]
 [8.597e+00 1.860e+01 5.409e+01 2.212e+02 1.074e-01 1.000e+00]
 [8.571e+00 1.310e+01 5.453e+01 2.213e+02 1.036e-01 1.000e+00]
 [8.598e+00 2.098e+01 5.466e+01 2.218e+02 1.243e-01 1.000e+00]
 [8.618e+00 1.179e+01 5.434e+01 2.245e+02 9.752e-02 1.000e+00]
 [8.671e+00 1.445e+01 5.442e+01 2.272e+02 9.138e-02 1.000e+00]
 [8.726e+00 1.583e+01 5.584e+01 2.309e+02 1.150e-01 1.000e+00]
 [8.734e+00 1.684e+01 5.527e+01 2.343e+02 1.039e-01 1.000e+00]
 [8.878e+00 1.549e+01 5.674e+01 2.410e+02 8.293e-02 1.000e+00]
 [8.888e+00 1.464e+01 5.879e+01 2.440e+02 9.783e-02 1.0

In [27]:
itr = 10
space = 1
col=[]
splitting(test,itr,header,space,col)


-> mean_radius < 16.25
-> mean_radius >= 16.25
---> mean_texture < 17.25
---> mean_texture >= 17.25
-----> mean_perimeter < 111.2
-----------------------------------------------------------------------------------


-----> mean_perimeter >= 111.2 ---> stop , predicted value is  0.0


 [[1.705e+01 1.908e+01 1.134e+02 8.950e+02 1.141e-01 0.000e+00]
 [1.702e+01 2.398e+01 1.128e+02 8.993e+02 1.197e-01 0.000e+00]
 [1.706e+01 2.100e+01 1.118e+02 9.186e+02 1.119e-01 0.000e+00]
 [1.746e+01 3.928e+01 1.134e+02 9.206e+02 9.812e-02 0.000e+00]
 [1.719e+01 2.207e+01 1.116e+02 9.283e+02 9.726e-02 0.000e+00]
 [1.727e+01 2.542e+01 1.124e+02 9.288e+02 8.331e-02 0.000e+00]
 [1.720e+01 2.452e+01 1.142e+02 9.294e+02 1.071e-01 0.000e+00]
 [1.708e+01 2.715e+01 1.112e+02 9.309e+02 9.898e-02 0.000e+00]
 [1.729e+01 2.213e+01 1.144e+02 9.478e+02 8.999e-02 0.000e+00]
 [1.742e+01 2.556e+01 1.145e+02 9.480e+02 1.006e-01 0.000e+00]
 [1.754e+01 1.932e+01 1.151e+02 9.516e+02 8.968e-02 0.000e+00]
 [1.768e+01 2.074e+0